In [ ]:
import os
import json
import requests
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

# Constants for API interaction
CSV_FILE = "chat_history.csv"
API_URL = "https://71d0-84-216-200-13.ngrok-free.app/v1/chat/completions"

# Function to get chat completion from the API
def get_chat_completion(url, user_prompt):
    payload = {
        "user_prompt": user_prompt,
    }

    json_payload = json.dumps(payload)
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(url, data=json_payload, headers=headers)
        response.raise_for_status()
        response_data = response.json()
        # Adjust according to your response structure
        return response_data["choices"][0]["message"]["content"]
    except requests.exceptions.HTTPError as http_err:
        return f"HTTP error occurred: {http_err}"
    except requests.exceptions.RequestException as req_err:
        return f"Error occurred: {req_err}"
    except KeyError as key_err:
        return f"Key error occurred: {key_err}"
    except Exception as err:
        return f"An error occurred: {err}"

# Create interactive widgets for the chatbot
input_area = widgets.Textarea(
    value='',
    placeholder='Message ChatGPT',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)

# Load chat history
def load_chat_history():
    if os.path.exists(CSV_FILE):
        return pd.read_csv(CSV_FILE)
    else:
        return pd.DataFrame(columns=["ChatID", "Role", "Content"])

chat_history_df = load_chat_history()

# Initialize chat session state
chat_names = chat_history_df["ChatID"].unique().tolist() if not chat_history_df.empty else ["Session 1"]
selected_chat = chat_names[0] if chat_names else "Session 1"

chat_select = widgets.Dropdown(options=chat_names, description='Chat Sessions:')
send_button = widgets.Button(description="Send")
output_area = widgets.Output(layout={'border': '1px solid black'})

# Function to save chat history to CSV
def save_chat_history(df):
    df.to_csv(CSV_FILE, index=False)

# Function to handle button click
def on_button_click(b):
    global chat_history_df, selected_chat

    question = input_area.value
    if not question.strip():
        return  # Don't do anything if the input is empty

    with output_area:
        # Display the user's question immediately
        display(HTML(f"<b>User:</b> {question}<br>"))

    # Fetch the assistant's response
    answer = get_chat_completion(API_URL, question)
    
    # Display the assistant's answer
    with output_area:
        display(HTML(f"<b>AI:</b> {answer}<br>" + "-"*50))

    # Save the chat history
    # chat_history_df = chat_history_df.append({
    #     "ChatID": selected_chat,
    #     "Role": "User",
    #     "Content": question
    # }, ignore_index=True)

    # chat_history_df = chat_history_df.append({
    #     "ChatID": selected_chat,
    #     "Role": "AI",
    #     "Content": answer
    # }, ignore_index=True)

    # save_chat_history(chat_history_df)

    # Clear the input box after submission
    input_area.value = ""

send_button.on_click(on_button_click)

# Display widgets
display(chat_select, input_area, send_button, output_area)

# Update chat session dropdown options and selected chat
def update_chat_session(change):
    global selected_chat
    selected_chat = chat_select.value

chat_select.observe(update_chat_session, names='value')
